# Here we will analyze the game data

### Preparing Data

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# reading data file
df = pd.read_csv('./games.csv')

In [ ]:
# checking data
display(df.head(), df.shape, df.columns)
df.info()

In [ ]:
# setting columns names to lower case
df.columns = [name for name in df.columns.str.lower()]

In [ ]:
# checking for duplicates
display(df.duplicated().sum())

---

In [ ]:
# checking for missing values ​​in name and gender column at the same time
display(df['name'].isna().sum())
display(df[df['name'].isna()].head())
display(df['genre'].isna().sum())
display(df[df['genre'].isna()].head())

# dropping rows with missing values in name and genre columns
df.dropna(subset=['name', 'genre'], inplace=True)

We removed the 2 lines with missing values ​​in name and genre, they seem to be 2 isolated cases that will not hinder our analysis.

---

In [ ]:
# checking for missing values on year_of_release column
display(df['year_of_release'].isna().sum())

# filling missing values with 0
df['year_of_release'] = df['year_of_release'].fillna(0)

# checking if can convert year column to int
if np.array_equal(df['year_of_release'], df['year_of_release'].astype(int)):
    df['year_of_release'] = df['year_of_release'].astype(int)

# checking for missing values on year_of_release column
display(df['year_of_release'].isna().sum())

We don't know the reason for the missing values ​​but we'll fill them with 0 so we can know when 0 it was missing

---

In [ ]:
# checking for missing values on critic_score column
display(df['critic_score'].isna().sum())

# filling missing values with -1
df['critic_score'] = df['critic_score'].fillna(-1)

# checking if can convert critic_score column to int
if np.array_equal(df['critic_score'], df['critic_score'].astype(int)):
    df['critic_score'] = df['critic_score'].astype(int)

# checking for missing values on critic_score column
display(df['critic_score'].isna().sum())

We don't know the reason for the missing values, but we'll fill them with -1, since 0 could be a grade that could easily be confused with real data.

---

In [ ]:
# checking for missing values on user_score column
display(df['user_score'].isna().sum())

# filling missing values with -1
df['user_score'] = df['user_score'].fillna(-1)
# replacing 'tbd' with -2
df['user_score'] = df['user_score'].replace('tbd', -2)

# checking if can convert user_score column to float
df['user_score'] = df['user_score'].astype(float)

# checking for missing values on user_score column
display(df['user_score'].isna().sum())

We don't know the reason for the missing values, but we'll fill them with -1, since 0 could be a grade that could easily be confused with real data and we will fill 'tdb' with -2.

---

In [ ]:
# filling missing values on rating column
df['rating'] = df['rating'].fillna('RP')

Filling missing rating values ​​with RP (Rating Pending)

---

Making a column with the total of sales for each game

In [ ]:
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

---
### Starting the analysis

Let's see the distribution of games released each year in our data

In [ ]:

df[df['year_of_release'] != 0]['year_of_release'].value_counts().sort_index().plot(kind='bar',
                                                       figsize=(15, 5),
                                                       title='Year of release distribution',
                                                       xlabel='Year of release',
                                                       ylabel='Number of games released')
plt.show()

We can see that there was a peak between 2008 and 2009 but then the number of launches fell again.

---

In [ ]:
# checking platforms with the highest total sales
platforms_top10_sales = df.groupby('platform')['total_sales'].sum().sort_values(ascending=False).head(10).index.tolist()
display(platforms_top10_sales)

In [ ]:
# showing sales distribution of top 10 platforms over the years
df.query('platform in @platforms_top10_sales and year_of_release != 0').groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack().plot(
    kind='bar',
    figsize=(15, 5),
    title='Top 10 platforms sales distribution',
    xlabel='Year of release',
    ylabel='Total sales',
    stacked=True)
plt.legend(title='Platform', loc='upper left')
plt.show()

We can see that PlayStation versions of games peaked in sales for about 4 to 6 years after their release and lost sales immediately after a new version of the platform was released, making sales of the old version almost non-existent after 2 years. We also see that sales of video games increased with the new platforms, XBox 360, DS and Wii between 2006 and 2010 as a wave, but after that PlayStation games regained their status as a near monopoly. Furthermore, sales of video games declined significantly after this wave, factors such as subscription sales model, digital platforms and global financial situation can be considered for this.

---

Since we are talking about technology and a relatively young and immediate audience, we will use data from the most recent 5 years to plan for the next year.

In [ ]:
df_recent_released = df.query('year_of_release in [2012, 2013, 2014, 2015, 2016]').reset_index(drop=True)

In [ ]:
# checking the top platforms with the highest total sales in recent years
top_5 = df_recent_released.groupby('platform')['total_sales'].sum().sort_values(ascending=False).index.tolist()[:5]

---

Let's check which platforms had the most sales in the last 5 years

In [ ]:
df_recent_released.groupby('platform')['total_sales'].sum().plot(
    kind='bar',
    x='platform',
    figsize=(15, 5),
    legend=False,
    xlabel='Platform',
    ylabel='Total sales')
plt.show()

In [ ]:
df_recent_released.query('platform in @top_5').groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack().plot(
    kind='line',
    figsize=(15, 5),
    title='Top 5 platforms sales time line',
    xlabel='Year of release',
    ylabel='Total sales')
plt.legend(title='Platform', bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
df_recent_released.query('platform not in @top_5').groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack().plot(
    kind='line',
    figsize=(15, 5),
    title='Other platforms sales time line',
    xlabel='Year of release',
    ylabel='Total sales')
plt.legend(title='Platform', bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In the graphs generated we could observe that throughout the period games from the PS3 and PS4 platforms led in sales followed by X360, 3DS and xOne, and observing their behavior over the years we noticed that game sales on all platforms are falling and in the last year we had PS4 in the lead, followed by xOne, 3DS, PS3 and X360, game sales on the other platforms have very low values ​​in the last year.

---

In [ ]:
sns.set_theme(rc={'figure.figsize':(15, 5)})
sns.boxplot(data=df_recent_released.query('platform in @top_5'), y='platform', x='total_sales')
plt.title('Top 5 platforms sales distribution')
plt.xlabel('Total sales')
plt.ylabel('Platform')
plt.show()

In this box plot we can't extract much useful information, but we can see that there are several outliers.

In [ ]:
df_recent_released.query('platform in @top_5').groupby('platform')['total_sales'].median().reset_index().plot(
    kind='bar',
    x='platform',
    figsize=(15, 5),
    legend=False,
    title='Top 5 platforms sales median',
    xlabel='Platform',
    ylabel='Total sales')
plt.show()

In [ ]:
df_recent_released.query('platform in @top_5').groupby('platform')['total_sales'].mean().reset_index().plot(
    kind='bar',
    x='platform',
    figsize=(15, 5),
    legend=False,
    title='Top 5 platforms sales mean',
    xlabel='Platform',
    ylabel='Total sales')
plt.show()

The differences in sales are quite significant, and the platform averages diverge from the medians due to the large number of outliers. Overall, we notice that sales of games for the Xbox 360 are much more significant than they initially seemed. We can assume that a few very successful games lead to this discrepancy between the values ​​and their medians.

---

In [ ]:
# filtering the top platform with the highest total sales in recent years
df_recent_released_top_seler_platform = df_recent_released.query('platform == @top_5[0]')

In [ ]:
# plotting the sales distribution by User score
df_recent_released_top_seler_platform.query('user_score >= 0').plot(
    kind='scatter',
    x='user_score',
    y='total_sales',
    figsize=(15, 5),
    title='Sales distribution by user score',
    xlabel='User score',
    ylabel='Total sales')
plt.show()

# showing the correlation between user score and total sales
display(df_recent_released_top_seler_platform.query('user_score >= 0')['user_score'].corr(df_recent_released_top_seler_platform['total_sales']))

In [ ]:
# plotting the sales distribution by Critic score
df_recent_released_top_seler_platform.query('critic_score >= 0').plot(
    kind='scatter',
    x='critic_score',
    y='total_sales',
    figsize=(15, 5),
    title='Sales distribution by Critic score',
    xlabel='Critic score',
    ylabel='Total sales')

plt.show()

# showing the correlation between critic score and total sales
display(df_recent_released_top_seler_platform.query('critic_score >= 0')['critic_score'].corr(df_recent_released_top_seler_platform['total_sales']))

Professional reviews have a greater correlation with sales than user reviews; user reviews can be negative in the case of the PS4, for example.

---

In [ ]:
# listing names of top 20 best selers games
top_seler_games_names = df_recent_released[['name', 'total_sales']].sort_values(by='total_sales', ascending=False).head(20).name.tolist()

# creating a new dataframe with the games in the list
df_games_to_compare = df_recent_released.query('name in @top_seler_games_names')

# grouping by name and platform, summing the total sales and unstacking to create a pivot table to plot
df_games_to_compare.groupby(['name', 'platform'])['total_sales'].sum().unstack().fillna(0).plot(
    kind='bar',
    stacked=True,
    figsize=(15, 10),
    title='Top 20 games total sales by platform'
    )
plt.ylabel('Total Sales (in millions)')
plt.xlabel('')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Platform')
plt.tight_layout()
plt.show()

We selected the top 20 best-selling games from 2012 to 2016 and stacked them in total sales by platform in a bar graph, so we can see how sales behaved on each platform for the same game.

---

In [ ]:
df_recent_released.groupby('genre')['total_sales'].sum().sort_values(ascending=False).plot(
    kind='bar',
    figsize=(15, 10),
    title='Total sales by genre'
    )
plt.ylabel('Total Sales (in millions)')
plt.xlabel('')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
sns.set_theme(rc={'figure.figsize':(15, 5)})
sns.boxplot(data=df_recent_released, x='total_sales', hue='genre', palette='Set2')
plt.title('Boxplot of total sales by genre')
plt.xlabel('Total Sales (in millions)')
plt.show()

In absolute sales values ​​by genre, we observe a difference with the distribution, which was already expected due to some isolated cases of great success of some games in relation to the majority. The most profitable genres would be Shooter, Platform, Sports, RP and Action, and we cannot generalize, because as already observed, some outliers are taking the absolute sales values ​​to the heights, but we know that this is not a pattern when we look at the distribution.

---

### Region profile

In [ ]:
# checking the top 5 platforms with the highest sales in North America
na_top5_platforms = df_recent_released.groupby('platform')['na_sales'].sum().sort_values(ascending=False)[:5].reset_index()

# checking the total sales in North America
na_total_sales = df_recent_released['na_sales'].sum()

# calculating the market share of each top 5 platforms in North America
na_top5_platforms['market_share'] = (na_top5_platforms['na_sales'] / na_total_sales * 100).round(2).astype(str) + '%'

display(na_top5_platforms)

In [ ]:
# checking the top 5 platforms with the highest sales in Europe
eu_top5_platforms = df_recent_released.groupby('platform')['eu_sales'].sum().sort_values(ascending=False)[:5].reset_index()

# checking the total sales in Europe
eu_total_sales = df_recent_released['eu_sales'].sum()

# calculating the market share of each top 5 platforms in Europe
eu_top5_platforms['market_share'] = (eu_top5_platforms['eu_sales'] / eu_total_sales * 100).round(2).astype(str) + '%'

display(eu_top5_platforms)

In [ ]:
# checking the top 5 platforms with the highest sales in North America
jp_top5_platforms = df_recent_released.groupby('platform')['jp_sales'].sum().sort_values(ascending=False)[:5].reset_index()

# checking the total sales in North America
jp_total_sales = df_recent_released['jp_sales'].sum()

# calculating the market share of each top 5 platforms in North America
jp_top5_platforms['market_share'] = (jp_top5_platforms['jp_sales'] / jp_total_sales * 100).round(2).astype(str) + '%'

display(jp_top5_platforms)

We calculated the market share for the main platforms in the 3 regions and were able to observe the differences in favorite platforms for each of them.

---

In [ ]:
# checking the top 5 genres with the highest sales in North America
na_top5_genres = df_recent_released.groupby('genre')['na_sales'].sum().sort_values(ascending=False)[:5].index.tolist()

display(na_top5_genres)

In [ ]:
# checking the top 5 genres with the highest sales in Europe
eu_top5_genres = df_recent_released.groupby('genre')['eu_sales'].sum().sort_values(ascending=False)[:5].index.tolist()

display(eu_top5_genres)

In [ ]:
# checking the top 5 genres with the highest sales in Japan
jp_top5_genres = df_recent_released.groupby('genre')['jp_sales'].sum().sort_values(ascending=False)[:5].index.tolist()

display(jp_top5_genres)

North America and Europe follow very similar genre preferences, while Japan has a very distinct preference.

---

In [ ]:
df_recent_released.groupby('rating')['na_sales'].sum().sort_values(ascending=False)

In [ ]:
df_recent_released.groupby('rating')['eu_sales'].sum().sort_values(ascending=False)

In [ ]:
df_recent_released.groupby('rating')['jp_sales'].sum().sort_values(ascending=False)

Once again, North America and Europe are similar while Japan is different, so yes, sales are affected in different regions according to the classification.

---

 ### Hypothesis testing

In [ ]:
# preparing data for the t-test
sample_xone_user_score = df_recent_released.query('user_score >= 0 and platform == "XOne"')['user_score']
sample_pc_user_score = df_recent_released.query('user_score >= 0 and platform == "PC"')['user_score']

alpha = 0.05
# H0: The average user score of XOne and PC platforms are equal
# H1: The average user score of XOne and PC platforms are not equal

# checking the sample distribution
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.histplot(sample_xone_user_score, kde=True, color='blue', label='XOne', bins=30)
plt.title('XOne User Score Distribution')
plt.xlabel('User Score')
plt.ylabel('Frequency')
plt.legend()
plt.subplot(1, 2, 2)
sns.histplot(sample_pc_user_score, kde=True, color='red', label='PC', bins=30)
plt.title('PC User Score Distribution')
plt.xlabel('User Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# checking the t-test
t_test = stats.ttest_ind(sample_xone_user_score, sample_pc_user_score, equal_var=False)

# checking the t-test result
if t_test.pvalue < alpha:
    print('The null hypothesis is rejected.')
else:
    print('The null hypothesis cannot be rejected.')

The null hypothesis cannot be rejected, so we have reason to believe that there is a great similarity between the distributions.

---

In [ ]:
sample_action_user_score = df_recent_released.query('user_score >= 0 and genre == "Action"')['user_score']
sample_sports_user_score = df_recent_released.query('user_score >= 0 and genre == "Sports"')['user_score']

alpha = 0.05
# H0: The average user score of Action and Sports genres are equal
# H1: The average user score of Action and Sports genres are not equal

# checking the sample distribution
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.histplot(sample_action_user_score, kde=True, color='blue', label='Action', bins=30)
plt.title('Action User Score Distribution')
plt.xlabel('User Score')
plt.ylabel('Frequency')
plt.legend()
plt.subplot(1, 2, 2)
sns.histplot(sample_sports_user_score, kde=True, color='red', label='Sports', bins=30)
plt.title('Sports User Score Distribution')
plt.xlabel('User Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# checking the t-test
t_test = stats.ttest_ind(sample_action_user_score, sample_sports_user_score, equal_var=False)

# checking the t-test result
if t_test.pvalue < alpha:
    print('The null hypothesis is rejected.')
else:
    print('The null hypothesis cannot be rejected.')

The null hypothesis was rejected, so we have reason to believe that there is no similarity between the distributions.

---

### Conclusion

During the analysis, we saw that game sales have fallen in recent years on all platforms, but PS4 and X360 lead the way. We noticed that professional reviews are more relevant than user reviews, and that PS4 has a high number of sales of specific games, while on average it is lower than others. The same is true for action games. GTA V is the game with the highest sales volume, surpassing 50 million dollars.

Europe and North America prefer Sony, while Japan prefers Nintendo. Japan also differs from other regions when it comes to genre preference and rating.

We confirmed that the average user ratings for the Xbox One and PC platforms are the same, and the average user ratings for the Action and Sports genres are different, both following an alpha threshold of 5%.

Now we can target advertising campaigns for each region, platform and genre, precisely to reach the right audience and obtain a greater volume of sales.